In [ ]:
import pickle
import pandas as pd
import numpy as np
import pickle
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import itertools
from collections import Counter
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string

import os
import numpy as np
from gensim.models import word2vec
import re


In [ ]:
# Load the preprocessed DataFrame from a Pickle file
with open('df_train_preprocessed.pkl', 'rb') as file:
    df_train = pickle.load(file)

with open('df_test_preprocessed.pkl', 'rb') as file:
    df_test = pickle.load(file)

In [ ]:
def build_vocab(sentences):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return word_counts, vocabulary, vocabulary_inv

In [ ]:
def get_embeddings(inp_data, vocabulary_inv, size_features=150,
                   mode='skipgram',
                   min_word_count=2,
                   context=9):
    model_name = "embedding"
    model_name = os.path.join(model_name)
    num_workers = 15  # Number of threads to run in parallel
    downsampling = 1e-3  # Downsample setting for frequent words
    print('Training Word2Vec model...')
    # use inp_data and vocabulary_inv to reconstruct sentences
    sentences = [[vocabulary_inv[w] for w in s] for s in inp_data]
    if mode == 'skipgram':
        sg = 1
        print('Model: skip-gram')
    elif mode == 'cbow':
        sg = 0
        print('Model: CBOW')
    embedding_model = word2vec.Word2Vec(sentences, workers=num_workers,
                                        sg=sg,
                                        vector_size=size_features,
                                        min_count=min_word_count,
                                        window=context,
                                        sample=downsampling)
    print("Saving Word2Vec model {}".format(model_name))
    embedding_weights = np.zeros((len(vocabulary_inv), size_features))
    for i in range(len(vocabulary_inv)):
        word = vocabulary_inv[i]
        if word in embedding_model.wv:
            embedding_weights[i] = embedding_model.wv[word]
        else:
            embedding_weights[i] = np.random.uniform(-0.25, 0.25,
                                                     embedding_model.vector_size)
    return embedding_weights

In [ ]:
# tokenization
tagged_data = [word_tokenize(_d) for i, _d in enumerate(df_train["text"])]
# build vocabulary from tokenized data
word_counts, vocabulary, vocabulary_inv = build_vocab(tagged_data)
# use the above mapping to create input data
inp_data = [[vocabulary[word] for word in text] for text in tagged_data]

In [ ]:
embedding_weights_2vec = get_embeddings(inp_data, vocabulary_inv)

In [ ]:
tagged_train_data = [word_tokenize(_d) for i, _d in enumerate(df_train["text"])]
tagged_test_data = [word_tokenize(_d) for i, _d in enumerate(df_test["text"])]

train_vec = []
for doc in tagged_train_data:
    vec = 0
    for w in doc:
        vec += embedding_weights_2vec [vocabulary[w]]
    vec = vec / len(doc)
    train_vec.append(vec)

import random
labels = df_train["label"].tolist()
combined = list(zip(train_vec, labels))

random.shuffle(combined)
shuffled_train_vec, shuffled_labels = zip(*combined)
split_size = int(len(shuffled_train_vec) * 0.8)
train_vec1 = shuffled_train_vec[:split_size]
valid_vec1 = shuffled_train_vec[split_size:]

train_labels1 = shuffled_labels[:split_size]
valid_labels1 = shuffled_labels[split_size:]

test_vec = []
for doc in tagged_test_data:
    vec = 0
    length = 0
    for w in doc:
        try:
            vec += embedding_weights_2vec [vocabulary[w]]
            length += 1
        except:
            continue
    vec = vec / length
    test_vec.append(vec)